In [0]:
CREATE OR REPLACE TABLE workspace.default.mimic_cxr_test_set_label_explanation_extract_gpt_5_1_v2
SELECT
  subject_id,
  study_id,
  findings,
  impression,
  get_json_object(ai_output, '$.label') AS label,
  get_json_object(ai_output, '$.explanation') AS explanation,
  ai_output AS raw_ai_output
FROM (
  SELECT
    subject_id,
    study_id,
    findings,
    impression,
    ai_query(
      'databricks-gpt-5-1',
      CONCAT(
        'Given the input as findings and impression, output ONLY a valid & strict JSON object within {} (and strictly not string) with two fields: label (a single word: Normal, Abnormal, or Uncertain) and explanation (a detailed reason matching the label). Do not include any other text. Findings: ', findings, '. Impression: ', impression
      ),
        modelParameters => named_struct('temperature', 0.8)
    ) AS ai_output
  FROM workspace.default.mimic_cxr_test_set
-- LIMIT 50
) t